In [4]:
# Look at a numpy file containing 3D keypoints

import numpy as np
from pathlib import Path

# ─── edit this to your target file ────────────────────────────────────────────
file_path = Path("Data-REHAB24-6/mp_keypoints/Ex6/PM_008-Camera17-30fps-mp.npy")
# ────────────────────────────────────────────────────────────────────────────────

# load
arr = np.load(file_path)

# basic info
print(f"Loaded: {file_path}")
print(f" dtype: {arr.dtype}")
print(f" shape: {arr.shape}  (frames × landmarks × coords)\n")

# show first few frames
n_show = min(3, arr.shape[0])
for i in range(n_show):
    print(f"Frame #{i:03d} (33×3):")
    print(arr[i])
    print(f"  → first landmark: {tuple(arr[i,0])}\n")

# overall statistics
print("Overall coordinate stats:")
for idx, name in enumerate(("x", "y", "z")):
    data = arr[..., idx]
    print(f"  {name}: min={data.min():.3f}, max={data.max():.3f}, mean={data.mean():.3f}")


Loaded: Data-REHAB24-6/mp_keypoints/Ex6/PM_008-Camera17-30fps-mp.npy
 dtype: float32
 shape: (5191, 33, 3)  (frames × landmarks × coords)

Frame #000 (33×3):
[[-0.03379065 -0.6112996  -0.22619084]
 [-0.02455677 -0.628223   -0.2275483 ]
 [-0.0257254  -0.6304051  -0.21732828]
 [-0.02547118 -0.6302204  -0.21820049]
 [-0.02778288 -0.6363151  -0.2358914 ]
 [-0.02677054 -0.63463634 -0.24706481]
 [-0.02263773 -0.61979294 -0.2281486 ]
 [ 0.02648694 -0.6184615  -0.16859515]
 [-0.03559308 -0.56201506 -0.14808732]
 [ 0.0030987  -0.59678274 -0.19447449]
 [-0.01712019 -0.55977213 -0.21580447]
 [ 0.1250833  -0.49333623 -0.08702794]
 [-0.05722423 -0.53108674 -0.02576461]
 [ 0.15178505 -0.51440114 -0.09251688]
 [-0.17247145 -0.4994753  -0.05377672]
 [ 0.14430666 -0.54461473 -0.03090633]
 [-0.32217076 -0.5845331  -0.08351779]
 [ 0.1352469  -0.5789426  -0.02217976]
 [-0.33411983 -0.62724733 -0.13406767]
 [ 0.11857966 -0.58959824 -0.04421883]
 [-0.30619952 -0.65674794 -0.14033641]
 [ 0.13962431 -0.533594

In [7]:
import pandas as pd
import numpy as np
import mediapipe as mp
import math
from pathlib import Path

# — 1. Helpers — 
def angle_between(a: np.ndarray, b: np.ndarray, c: np.ndarray) -> float:
    """Return ∠ABC in degrees."""
    BA = a - b
    BC = c - b
    cosθ = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))
    return math.degrees(math.acos(np.clip(cosθ, -1.0, 1.0)))

# — 2. Define which 3 landmarks form each joint angle —
PoseLandmark = mp.solutions.pose.PoseLandmark
JOINT_TRIPLETS = {
    "LEFT_ELBOW":   (PoseLandmark.LEFT_SHOULDER.value,
                     PoseLandmark.LEFT_ELBOW.value,
                     PoseLandmark.LEFT_WRIST.value),
    "RIGHT_ELBOW":  (PoseLandmark.RIGHT_SHOULDER.value,
                     PoseLandmark.RIGHT_ELBOW.value,
                     PoseLandmark.RIGHT_WRIST.value),

    "LEFT_SHOULDER":  (PoseLandmark.LEFT_ELBOW.value,
                       PoseLandmark.LEFT_SHOULDER.value,
                       PoseLandmark.LEFT_HIP.value),
    "RIGHT_SHOULDER": (PoseLandmark.RIGHT_ELBOW.value,
                       PoseLandmark.RIGHT_SHOULDER.value,
                       PoseLandmark.RIGHT_HIP.value),

    "LEFT_HIP":   (PoseLandmark.LEFT_SHOULDER.value,
                   PoseLandmark.LEFT_HIP.value,
                   PoseLandmark.LEFT_KNEE.value),
    "RIGHT_HIP":  (PoseLandmark.RIGHT_SHOULDER.value,
                   PoseLandmark.RIGHT_HIP.value,
                   PoseLandmark.RIGHT_KNEE.value),

    "LEFT_KNEE":  (PoseLandmark.LEFT_HIP.value,
                  PoseLandmark.LEFT_KNEE.value,
                  PoseLandmark.LEFT_ANKLE.value),
    "RIGHT_KNEE": (PoseLandmark.RIGHT_HIP.value,
                  PoseLandmark.RIGHT_KNEE.value,
                  PoseLandmark.RIGHT_ANKLE.value),

    # approximate “vertical” spine / head angles
    "SPINE": (PoseLandmark.LEFT_SHOULDER.value,
              PoseLandmark.LEFT_HIP.value,
              PoseLandmark.LEFT_HIP.value),
    "HEAD":  (PoseLandmark.LEFT_SHOULDER.value,
              PoseLandmark.NOSE.value,
              PoseLandmark.LEFT_SHOULDER.value),
}

# — 3. Paths & load metadata —
DATA_ROOT = Path("Data-REHAB24-6")
KEYPT_ROOT = DATA_ROOT / "mp_keypoints"
META_FILE  = DATA_ROOT / "Segmentation.xlsx"

df = pd.read_excel(META_FILE, engine="openpyxl")
df = df.sort_values(["video_id","repetition_number"]).reset_index(drop=True)

# — 4. Compute data‐driven “ideal_angles” from correct reps —
ideal_angles: dict[int, dict[str,float]] = {}

correct = df[df.correctness == 1]

for ex in sorted(correct.exercise_id.unique()):
    # collect one angle measurement per correct repetition
    joint_angles: dict[str, list[float]] = {jn: [] for jn in JOINT_TRIPLETS}

    for _, row in correct[correct.exercise_id == ex].iterrows():
        vid = row.video_id
        first, last = int(row.first_frame), int(row.last_frame)
        files = list((KEYPT_ROOT/f"Ex{ex}").glob(f"{vid}-Camera17*-mp.npy"))
        if not files:
            continue

        arr = np.load(files[0])  # shape (F_all, N_JOINTS, 3)
        segment = arr[first:last] if last>first else arr[first:]
        if segment.size == 0:
            continue

        # pick the middle frame in that segment
        mid = segment.shape[0] // 2
        frame_xyz = segment[mid]  # (N_JOINTS, 3)

        # measure each joint angle
        for jn, (ia, ib, ic) in JOINT_TRIPLETS.items():
            a = frame_xyz[ia, :2]
            b = frame_xyz[ib, :2]
            c = frame_xyz[ic, :2]
            ang = angle_between(a, b, c)
            joint_angles[jn].append(ang)

    # take median for stability
    ideal_angles[ex] = {
        jn: float(np.median(joint_angles[jn]))
        for jn in joint_angles if joint_angles[jn]
    }

print("Computed ideal_angles:")
for ex, m in ideal_angles.items():
    print(f" Ex{ex}:")
    for jn, ang in m.items():
        print(f"   {jn:>14s} → {ang:.1f}°")


# — 5. Now compute err_0…err_32 for *every* row using those ideals —
n_joints = len(JOINT_TRIPLETS)
all_errs = np.zeros((len(df), n_joints), dtype=np.float32)
joint_names = list(JOINT_TRIPLETS.keys())

for i, row in df.iterrows():
    vid = row.video_id
    ex  = int(row.exercise_id)
    first, last = int(row.first_frame), int(row.last_frame)
    files = list((KEYPT_ROOT/f"Ex{ex}").glob(f"{vid}-Camera17*-mp.npy"))
    if not files:
        continue

    arr = np.load(files[0])
    segment = arr[first:last] if last>first else arr[first:]
    if segment.size == 0:
        continue

    # for each joint, compute mean observed angle over the entire segment
    for j, jn in enumerate(joint_names):
        ia, ib, ic = JOINT_TRIPLETS[jn]
        angs = [
            angle_between(f[x,:2], f[y,:2], f[z,:2])
            for f in segment
            for (x,y,z) in [(ia,ib,ic)]
        ]
        mean_obs = float(np.mean(angs))
        ideal = ideal_angles[ex].get(jn, mean_obs)
        all_errs[i, j] = mean_obs - ideal

# inject into df
for j, jn in enumerate(joint_names):
    df[f"err_{j}"] = all_errs[:, j]

# — 6. Save augmented metadata —
OUT = DATA_ROOT / "Segmentation_with_errs.xlsx"
df.to_excel(OUT, index=False)
print(f"✅ Written augmented metadata with err_0…err_{n_joints-1} to\n   {OUT}")


/var/folders/pv/99z70cjs7t1dx16_0r1c33hw0000gn/T/ipykernel_19661/1132478303.py:12: RuntimeWarning: invalid value encountered in scalar divide
  cosθ = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))


Computed ideal_angles:
 Ex1:
       LEFT_ELBOW → 170.8°
      RIGHT_ELBOW → 150.8°
    LEFT_SHOULDER → 21.4°
   RIGHT_SHOULDER → 119.8°
         LEFT_HIP → 166.3°
        RIGHT_HIP → 176.9°
        LEFT_KNEE → 177.4°
       RIGHT_KNEE → 169.6°
            SPINE → nan°
             HEAD → 0.0°
 Ex2:
       LEFT_ELBOW → 76.0°
      RIGHT_ELBOW → 42.3°
    LEFT_SHOULDER → 55.8°
   RIGHT_SHOULDER → 34.4°
         LEFT_HIP → 168.1°
        RIGHT_HIP → 176.7°
        LEFT_KNEE → 175.9°
       RIGHT_KNEE → 168.6°
            SPINE → nan°
             HEAD → 0.0°
 Ex3:
       LEFT_ELBOW → 106.4°
      RIGHT_ELBOW → 90.3°
    LEFT_SHOULDER → 20.8°
   RIGHT_SHOULDER → 11.8°
         LEFT_HIP → 161.0°
        RIGHT_HIP → 164.2°
        LEFT_KNEE → 173.9°
       RIGHT_KNEE → 169.9°
            SPINE → nan°
             HEAD → 0.0°
 Ex4:
       LEFT_ELBOW → 158.9°
      RIGHT_ELBOW → 131.0°
    LEFT_SHOULDER → 25.7°
   RIGHT_SHOULDER → 28.8°
         LEFT_HIP → 158.4°
        RIGHT_HIP → 165.2°
   

In [ ]:
# generate_windows.py
import pandas as pd
import numpy as np
import mediapipe as mp
import math
from pathlib import Path

# 1. helpers --------------------------------------------------
def angle_between(a,b,c):
    BA = a-b; BC = c-b
    cosθ = np.dot(BA,BC)/(np.linalg.norm(BA)*np.linalg.norm(BC))
    return math.degrees(math.acos(np.clip(cosθ,-1,1)))

PoseLandmark = mp.solutions.pose.PoseLandmark
JOINT_TRIPLETS = {
    "LEFT_ELBOW":   (PoseLandmark.LEFT_SHOULDER.value,
                     PoseLandmark.LEFT_ELBOW.value,
                     PoseLandmark.LEFT_WRIST.value),
    "RIGHT_ELBOW":  (PoseLandmark.RIGHT_SHOULDER.value,
                     PoseLandmark.RIGHT_ELBOW.value,
                     PoseLandmark.RIGHT_WRIST.value),
    "LEFT_SHOULDER":  (PoseLandmark.LEFT_ELBOW.value,
                       PoseLandmark.LEFT_SHOULDER.value,
                       PoseLandmark.LEFT_HIP.value),
    "RIGHT_SHOULDER": (PoseLandmark.RIGHT_ELBOW.value,
                       PoseLandmark.RIGHT_SHOULDER.value,
                       PoseLandmark.RIGHT_HIP.value),
    "LEFT_HIP":   (PoseLandmark.LEFT_SHOULDER.value,
                   PoseLandmark.LEFT_HIP.value,
                   PoseLandmark.LEFT_KNEE.value),
    "RIGHT_HIP":  (PoseLandmark.RIGHT_SHOULDER.value,
                   PoseLandmark.RIGHT_HIP.value,
                   PoseLandmark.RIGHT_KNEE.value),
    "LEFT_KNEE":  (PoseLandmark.LEFT_HIP.value,
                  PoseLandmark.LEFT_KNEE.value,
                  PoseLandmark.LEFT_ANKLE.value),
    "RIGHT_KNEE": (PoseLandmark.RIGHT_HIP.value,
                  PoseLandmark.RIGHT_KNEE.value,
                  PoseLandmark.RIGHT_ANKLE.value),
    "SPINE": (
       PoseLandmark.LEFT_HIP.value,       
       PoseLandmark.LEFT_SHOULDER.value,   
       PoseLandmark.RIGHT_SHOULDER.value   
    ),
    "HEAD": (
       PoseLandmark.LEFT_SHOULDER.value,
       PoseLandmark.NOSE.value,
       PoseLandmark.RIGHT_SHOULDER.value
    ),
}
ERR_JOINTS = list(JOINT_TRIPLETS.keys())
N_ERR = len(ERR_JOINTS)  # 10

# 2. load original metadata & keypoints -----------------------
DATA_ROOT    = Path("Data-REHAB24-6")
KEYPT_ROOT   = DATA_ROOT/"mp_keypoints"
META_ORIG    = DATA_ROOT/"Segmentation.xlsx"
df           = pd.read_excel(META_ORIG, engine="openpyxl")
df.columns   = df.columns.str.strip()

# 3. compute ideal_angles on correct reps ----------------------
ideal_angles = {}
correct = df[df.correctness==1]
for ex in correct.exercise_id.unique():
    all_ang = {jn:[] for jn in ERR_JOINTS}
    for _,r in correct[correct.exercise_id==ex].iterrows():
        vid, f0, f1 = r.video_id, int(r.first_frame), int(r.last_frame)
        files = list((KEYPT_ROOT/f"Ex{ex}").glob(f"{vid}-Camera17*-mp.npy"))
        if not files: continue
        arr = np.load(files[0])
        seg = arr[f0:f1] if f1>f0 else arr[f0:]
        if len(seg)==0: continue
        mid = len(seg)//2
        frm = seg[mid]
        for jn in ERR_JOINTS:
            ia,ib,ic = JOINT_TRIPLETS[jn]
            ang = angle_between(frm[ia,:2],frm[ib,:2],frm[ic,:2])
            all_ang[jn].append(ang)
    # median
    ideal_angles[ex] = {jn:float(np.median(all_ang[jn])) for jn in all_ang if all_ang[jn]}

# 4. slide windows & write rows --------------------------------
WINDOW, STRIDE = 16, 8
rows = []
for _,r in df.iterrows():
    vid, ex, f0, f1 = r.video_id, int(r.exercise_id), int(r.first_frame), int(r.last_frame)
    files = list((KEYPT_ROOT/f"Ex{ex}").glob(f"{vid}-Camera17*-mp.npy"))
    if not files: continue
    arr = np.load(files[0])                # (F,33,3)
    seg = arr[f0:f1] if f1>f0 else arr[f0:]
    if len(seg)<WINDOW: continue

    # per-frame errors
    pf_err = {jn:[] for jn in ERR_JOINTS}
    for frm in seg:
        for jn in ERR_JOINTS:
            ia,ib,ic = JOINT_TRIPLETS[jn]
            ang = angle_between(frm[ia,:2],frm[ib,:2],frm[ic,:2])
            pf_err[jn].append(ang - ideal_angles[ex].get(jn,ang))

    # slide
    for start in range(0, len(seg)-WINDOW+1, STRIDE):
        w = np.array([ pf_err[jn][start:start+WINDOW] for jn in ERR_JOINTS ])  # (10,WINDOW)
        mean_err = w.mean(axis=1)
        row = {
            "video_id":vid,
            "exercise_id":ex,
            "repetition_number":r.repetition_number,
            "window_start": f0+start,
            "window_end":   f0+start+WINDOW,
            "correctness":  r.correctness
        }
        for i,jn in enumerate(ERR_JOINTS):
            row[f"err_{i}"] = float(mean_err[i])
        rows.append(row)

win_df = pd.DataFrame(rows)
win_df.to_csv(DATA_ROOT/"Segmentation_windows.csv", index=False)
print("Wrote", len(win_df), "windows to Segmentation_windows.csv")


/var/folders/pv/99z70cjs7t1dx16_0r1c33hw0000gn/T/ipykernel_19661/669420515.py:11: RuntimeWarning: invalid value encountered in scalar divide
  cosθ = np.dot(BA,BC)/(np.linalg.norm(BA)*np.linalg.norm(BC))


Wrote 14375 windows to Segmentation_windows.csv
